<a href="https://colab.research.google.com/github/dvarelaj/nlp-miniproyecto-icesi/blob/main/analisis_sentimientos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Ohtar10/icesi-nlp.git

Cloning into 'icesi-nlp'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 196 (delta 78), reused 72 (delta 72), pack-reused 104 (from 2)
Receiving objects: 100% (196/196), 4.32 MiB | 8.81 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [29]:
import pandas as pd
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from spacy.matcher import Matcher
from collections import Counter
import spacy.cli
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [23]:
# URL de GitHub
url_datos = 'https://raw.githubusercontent.com/dvarelaj/nlp-miniproyecto-icesi/main/quejas_anonimizadas_muestra_etiqueta.csv'
df = pd.read_csv(url_datos, delimiter=';')

Luego, hagamos algo de limpieza, vamos a remover nulos y valores vacíos:

In [24]:
# Limpieza básica
df.dropna(subset=['descripcion_anonimizada'], inplace=True)
df = df[df['descripcion_anonimizada'].str.strip() != '']

print(f"Total de quejas para analizar: {len(df)}")

Total de quejas para analizar: 101


En lugar de nltk.vader, usaremos un analizador profesional para español ya que nuestro archivo de datos tiene quejas y peticiones en español.

In [25]:
#!pip install pysentimiento

In [26]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

# Aplicamos el análisis a nuestras quejas
df['sentiment_results'] = df['descripcion_anonimizada'].apply(lambda x: analyzer.predict(x))
df['prediction'] = df['sentiment_results'].apply(lambda r: r.output)
df['compound_score'] = df['sentiment_results'].apply(lambda r: r.probas['NEG'] * -1 if r.output == 'NEG' else r.probas['POS'])

df[['descripcion_anonimizada', 'prediction']].head()

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

,descripcion_anonimizada,prediction
0,usuaria se comunica porque requiere ingresar a...,NEG
1,empresa cc : [ID_ANONIMIZADO] ingresa con usua...,NEG
2,se comunica porque esta ingresando al portal p...,NEU
3,nehcesito afiliar a unas personas pero me sale...,NEG
4,id chat : [ID_ANONIMIZADO] canal de atencion w...,NEG


In [27]:
# Conteo de los resultados obtenidos por el modelo
conteo_sentimientos = df['prediction'].value_counts()
print(conteo_sentimientos)

prediction
NEG    71
NEU    30
Name: count, dtype: int64


In [36]:
# Predicciones del modelo
y_pred = [p.lower() for p in df['prediction'].tolist()]

# Obtener las etiquetas verdaderas
y_true = df['y_true'].tolist()

# Generar el reporte (requires y_true)
reporte = classification_report(y_true, y_pred)
print("### Reporte de Clasificación ###")
print(reporte)

# Calcular exactitud total (requires y_true)
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy Total: {accuracy:.2f}")

### Reporte de Clasificación ###
              precision    recall  f1-score   support

         neg       0.86      0.76      0.81        80
         neu       0.37      0.52      0.43        21

    accuracy                           0.71       101
   macro avg       0.61      0.64      0.62       101
weighted avg       0.76      0.71      0.73       101

Accuracy Total: 0.71
